In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from numpy.random import RandomState

from src.extraction.jsonl_data_reader import JsonlDataReader

In [2]:
seed = 7
random_state = RandomState(seed=seed)
np.random.seed(seed)

In [3]:
train_data = JsonlDataReader(file_name='train.jsonl').read()
test_data = JsonlDataReader(file_name='test.jsonl').read()

In [4]:
from src.preprocessing.simple_preprocessor import SimplePreprocessor

preprocessor = SimplePreprocessor(remove_citations=True, remove_duplicates=True)
preprocessed_train = preprocessor.preprocess(train_data)
preprocessed_test = preprocessor.preprocess(test_data)

In [5]:
from src.tokenize.spacy_tokenizer import SpacyTokenizer

tokenizer = SpacyTokenizer(replace_numbers=True)
tokenized_train = tokenizer.tokenize(train_data)
tokenized_test = tokenizer.tokenize(test_data)

In [28]:
from src.tokenize.sentence_piece_tokenizer import SentencePieceTokenizer
tokenizer = SentencePieceTokenizer(vocab_size=3000)
tokenizer.fit(preprocessed_train)
tokenized_train = tokenizer.tokenize(preprocessed_train)
tokenized_test = tokenizer.tokenize(preprocessed_test)

In [5]:
from src.tokenize.null_tokenizer import NullTokenizer

tokenizer = NullTokenizer()
tokenized_train = tokenizer.tokenize(preprocessed_train)
tokenized_test = tokenizer.tokenize(preprocessed_test)

In [13]:
from src.post_tokenizer.phraser_merger import PhraserMerger

post_tokenizer = PhraserMerger(num_gram=2)
tokenized_train = post_tokenizer.fit_transform(tokenized_train)
tokenized_test = post_tokenizer.transform(tokenized_test)

In [ ]:
from src.vectorizer.fasttext_w2v_vectorizer import FastTextW2vVectorizer

vectorizer = FastTextW2vVectorizer()
vectorizer.fit(tokenized_train)
vectorized_train = vectorizer.transform(tokenized_train)
vectorized_test = vectorizer.transform(tokenized_test)

In [7]:
from src.vectorizer.w2v_vectorizer import W2vVectorizer

vectorizer = W2vVectorizer(vector_size=300)
vectorizer.fit(tokenized_train)
vectorized_train = vectorizer.transform(tokenized_train)
vectorized_test = vectorizer.transform(tokenized_test)

In [6]:
from src.vectorizer.sk_count_vectorizer import SkCountVectorizer

vectorizer = SkCountVectorizer(ngram_range=(1, 2), ignore_preprocessing=False)
vectorizer.fit(tokenized_train)
vectorized_train = vectorizer.transform(tokenized_train)
vectorized_test = vectorizer.transform(tokenized_test)

In [6]:
from src.vectorizer.tfidf_vectorizer import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(tokenized_train)
vectorized_train = vectorizer.transform(tokenized_train)
vectorized_test = vectorizer.transform(tokenized_test)

In [7]:
from src.vectorizer.lsi_vectorizer import LsiVectorizer

vectorizer = LsiVectorizer(num_topics=100)
vectorizer.fit(tokenized_train)
vectorized_train = vectorizer.transform(tokenized_train)
vectorized_test = vectorizer.transform(tokenized_test)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.svm import SVC


model = SVC(C=10.0, kernel='rbf')
model.fit(vectorized_train.vectors, vectorized_train.label_indices)
y_pred_train = model.predict(vectorized_train.vectors)
training_score = f1_score(vectorized_train.label_indices, y_pred_train, average='macro')
print(f'{training_score=}')

y_pred_test = model.predict(vectorized_test.vectors)
testing_score = f1_score(vectorized_test.label_indices, y_pred_test, average='macro')
print(f'{testing_score=}')

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix

model = LogisticRegression()
model.fit(vectorized_train.vectors, vectorized_train.label_indices)
y_pred_train = model.predict(vectorized_train.vectors)
training_score = f1_score(vectorized_train.label_indices, y_pred_train, average='macro')
print(f'{training_score=}')
display(confusion_matrix(vectorized_train.label_indices, y_pred_train))

y_pred_test = model.predict(vectorized_test.vectors)
testing_score = f1_score(vectorized_test.label_indices, y_pred_test, average='macro')
print(f'{testing_score=}')
display(confusion_matrix(vectorized_test.label_indices, y_pred_test))

training_score=0.9998007816475948


array([[4320,    0,    0],
       [   0, 2191,    0],
       [   1,    0, 1037]], dtype=int64)

testing_score=0.8084549693024973


array([[894,  55,  48],
       [132, 455,  18],
       [ 52,  10, 197]], dtype=int64)

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

model = RandomForestClassifier(n_estimators=2000, max_depth=100, min_samples_leaf=5, n_jobs=-1)
model.fit(vectorized_train.vectors, vectorized_train.label_indices)
y_pred_train = model.predict(vectorized_train.vectors)
training_score = f1_score(vectorized_train.label_indices, y_pred_train, average='macro')
print(f'{training_score=}')

y_pred_test = model.predict(vectorized_test.vectors)
testing_score = f1_score(vectorized_test.label_indices, y_pred_test, average='macro')
print(f'{testing_score=}')

training_score=0.585507207014784
testing_score=0.5525111683691327
